# **Value at Risk(VaR)**

VaR measures the amount of potential loss that could happen in an investment over a given period of time(with a given degree of confidence).

As we know that stock prices follows a geometric brownian motion

$dS = S_0 \left(\mu dt + \sigma dW_t \right)$

Where W is a weiner process with mean 0 and variance t

$\mu$ is mean (drift) 

$\sigma$ is volatility

$S_0$ is  initial stock price






$dS = S_0 \left(\mu dt + \sigma dW_t \right)$

Solution of this stochastic differential equation is 


$$
\begin{equation*} 
    S_t = S_{0} e^{\left( \mu - \frac{\sigma^2 }{2} \right) dt + \sigma \sqrt{t} N(0,1)}
\end{equation*} 
$$


In Monte-Carlo Simulation we generate a large amount of stock price estimates with this equation.

Then we sort the stock prices form smallest to largest and if we need the **VaR** with 99% confidence level: we need the 1%
lowest percentile $S'(t)$ in this series because there is 1% chance that S(t) could fall to $S'(t)$ or less


$VaR($99$) = S(t) - S'(t)$



In [1]:
import numpy as np
import yfinance as yf
import datetime
import pandas as pd

In [2]:
def download_stock(stock,start,end):
    data={}
    ticker = yf.download(stock,start,end)
    data["Adj Close"] = ticker["Adj Close"]
    return pd.DataFrame(data)

In [3]:
# S is initial investment
class VaRMonteCarlo:
    
    def __init__(self,S,mu,sigma,c,n,iterations):
        self.S = S
        self.mu = mu
        self.sigma = sigma
        self.c = c
        self.n = n
        self.iterations = iterations
        
    def simulation(self):
        # generating weiner process
        rand = np.random.normal(0,1,[1,self.iterations])
        # equation for the S(t) of the stock price
        stock_price = self.S * np.exp(self.n*(self.mu - 0.5 * self.sigma **2)+ self.sigma * np.sqrt(self.n)*rand)
        # we have to sort the stock price to determine the percentile
        stock_price = np.sort(stock_price)
        # it depends on the confidence level : 95% -> 5 and 99% -> 1
        percentile = np.percentile(stock_price,(1-self.c)*100)
        return self.S -percentile
        
        
        
        

In [4]:
S = 100000 # Investment Amount
c = 0.95   # Confidence Level
n = 1 #day 
iterations = 10000 # no of paths in monte-carlo simulation

In [5]:
start_date = datetime.datetime(2020,1,1)
end_date = datetime.datetime(2022,1,1)
Reliance = download_stock("Reliance.NS",start_date,end_date)
Reliance["Returns"] = Reliance["Adj Close"].pct_change()

[*********************100%***********************]  1 of 1 completed


In [6]:
Reliance

,Adj Close,Returns
Date,,
2020-01-01,1480.595825,NaN
2020-01-02,1505.802124,0.017024
2020-01-03,1507.616577,0.001205
2020-01-06,1472.651367,-0.023192
2020-01-07,1495.307617,0.015385
...,...,...
2021-12-27,2363.135254,-0.001075
2021-12-28,2391.200684,0.011876
2021-12-29,2395.288330,0.001709


In [7]:
mu = np.mean(Reliance["Returns"])
sigma = np.std(Reliance["Returns"])

In [8]:
mu,sigma

(0.0012211383996952787, 0.02386203590328131)

In [9]:
Var_model = VaRMonteCarlo(S,mu,sigma,c,n,iterations)
simulation = Var_model.simulation()
simulation

3711.7418172263715

In [12]:
print(f"Value at Risk with Monte Carlo Simulation is Rs.{simulation} on investment of Rs.{S} for next {1} day.")

Value at Risk with Monte Carlo Simulation 3711.7418172263715 on investment of Rs.100000 for next 1 day.
